In [9]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5] #Plotgröße anpassen
import matplotlib
matplotlib.rcParams['text.usetex'] = True
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 18}
matplotlib.rc('font', **font)
from sympy import *
init_printing(use_latex='mathjax')
from scipy import signal
from IPython.display import display, Markdown

Rs,Ls,Cs,w,Z_L,Gs=symbols('R´,L´,C´,\omega,Z_L,G´',real=true)
j = I
Ur,Uh=symbols(r'\underline{U}_r,\underline{U}_h',real=false)

# Leitungstheorie
 {cite}`meinke2`

In der Leitungstheorie ist der Begin der Leitung Links. Also vom Verbraucher zum Generator hin. Strom und Spannung werden in Anhängigkeit der Position auf der Leitung $l$ angegeben.
```{figure} ./images/Leitung.jpg
:name: Leitung
:width: 400px

Strom und Spannung auf einer Leitung
```

Die Leitung kann in kleine Leitungsteile zerlegt werden.
```{figure} ./images/Leitungssegment.jpg
:name: Leitungssegment
:width: 400px

Leitungssegment
```

## Begriffe und Definitionen

| Symbol | Beschreibung  |
|--------|---:|
| $l$    |  Position auf der Leitung |
| $\underline{U}(l)$ | Spannung an der Position $l$ auf der Leitung|
| $\underline{I}(l)$ | Strom an der Position $l$ auf der Leitung|
| $\underline{U}_h(l,t)$ | Spannung an der Position $l$ zum Zeitpunkt $t$ vom Generator zum Verbraucher laufend, $h$ für hin|
| $\underline{I}_h(l,t)$ | Strom an der Position $l$ zum Zeitpunkt $t$ vom Generator zum Verbraucher laufend, $h$ für hin|
| $\underline{U}_r(l,t)$ | Spannung an der Position $l$ zum Zeitpunkt $t$ vom Generator zum Verbraucher laufend, $r$ für rück|
| $\underline{I}_r(l,t)$ | Strom an der Position $l$ zum Zeitpunkt $t$ vom Generator zum Verbraucher laufend, $r$ für rück|
| $U_1$ | Spannung am Leitungseingang|
| $U_2$ | Spannung am Leiungsausgang $U_2 = U(l=0)$|
| $R'$| Widerstandsbelag in $\frac{\Omega}{m}$|
| $L'$| Induktionsbelag in $\frac{H}{m}$|
| $G'$| Ableitungsbelag in $\frac{S}{m}$|
| $C'$| Kapazitätsbelag in $\frac{F}{m}$|
|$Z_L$| Leitungswiderstand|
|$Z_2$| Lastwiderstand|

Die Werte der Leitungsbeläge müssen durch Messung bestimmt werden.

### Leitungswiderstand

In [8]:
eq1 = Eq(Z_L,sqrt((Rs+j*w*Ls)/(Gs+j*w*Cs)))
display(eq1)

          __________________
         ╱ ⅈ⋅L´⋅\omega + R´ 
Z_L =   ╱  ──────────────── 
      ╲╱   ⅈ⋅C´⋅\omega + G´ 

### Reflexionsfaktor

In [11]:
eq2=Eq(S('r'),Ur/Uh)
display(eq2)

    \underline{U}ᵣ
r = ──────────────
    \underline{U}ₕ

## Leitungsabschlüsse

### Beliebiger Abschluss